In [1]:
import json
from IPython.display import clear_output, HTML, display
import ipywidgets.widgets as widgets
from tqdm.notebook import tqdm
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
from io import BytesIO
from copy import deepcopy

import sys
sys.path.append('/home/ballsbot/projects/ballsbot/python/lib')

from ballsbot.utils import figsize_from_image_size
from ballsbot.lidar import apply_transformation_to_cloud, revert_transformation_to_cloud
import ballsbot.config as config
config.ENABLE_MULTIPROCESSING = False

In [2]:
new_format = True
with open('track_2024-08-20.json', 'r') as hf:
    if new_format:
        track = []
        for line in hf:
            try:
                frame = json.loads(line)
                track.append(frame)
            except Exception as e:
                print(f'broken frame {e}')
    else:
        track = json.loads(hf.read())

In [3]:
print(len(track))

516


In [4]:
import math

# first_i = 100
# second_i = 105

# first_i = 248
# second_i = 250

# first_i = 101
# second_i = 102

first_i = 4
second_i = 5

# first_i = 121
# second_i = 122

first_frame = track[first_i]
second_frame = track[second_i]

# first_frame['tracker']['current_pose'] = {'x': 1., 'y': 2., 'teta': math.radians(15), 'ts': 0.}
# second_frame['tracker']['current_pose'] = {'x': 2., 'y': 7., 'teta': math.radians(45), 'ts': 1.}

# first_frame['tracker']['free_tile_centers'] = None
# second_frame['tracker']['free_tile_centers'] = None

second_frame['lidar']['augmented_points'] = deepcopy(first_frame['lidar']['augmented_points'])
second_frame['tracker']['current_pose'] = {'x': 1., 'y': 2., 'teta': 3., 'ts': second_frame['tracker']['current_pose']['ts']}
first_frame['tracker']['current_pose'] = {'x': 1., 'y': 2., 'teta': 3., 'ts': first_frame['tracker']['current_pose']['ts']}

second_frame['lidar']['augmented_points'] = revert_transformation_to_cloud(
    second_frame['lidar']['augmented_points'],
    [0.1, 0.15, 0.05]
)

print(first_frame['tracker']['current_pose'])
print(second_frame['tracker']['current_pose'])

{'x': 1.0, 'y': 2.0, 'teta': 3.0, 'ts': 1724168571.5121946}
{'x': 1.0, 'y': 2.0, 'teta': 3.0, 'ts': 1724168572.3364182}


In [5]:
import ballsbot.drawing as drawing
from ballsbot.lidar import calibration_to_xywh
from ballsbot.drawing import BotPoseAbsCoordsDrawing
from ballsbot.utils import keep_rps
from ballsbot.lidar import revert_transformation_to_cloud, apply_transformation_to_cloud
from ballsbot.dashboard import Dashboard

In [6]:
dashboard = Dashboard(widgets, image_size=1200)

augmented_painter = BotPoseAbsCoordsDrawing(dashboard, 'augmented')
dashboard.add_subplot('augmented', augmented_painter.get_drawing_func_name())
sparse_painter = BotPoseAbsCoordsDrawing(dashboard, 'sparse')
dashboard.add_subplot('sparse', sparse_painter.get_drawing_func_name())

display(dashboard.get_image())

Image(value=b'', format='jpeg', height='1200', width='1200')

In [7]:
only_nearby_meters = 5
calibration = {
    'angle_fix': 3.0810609316737434,  # lidar orientation (radians)
    'fl_x': 0.2041686441507201,  # front left corner coords
    'fl_y': 0.10426277741236079,
    'rr_x': -0.08011094659163859,  # rear right corner coords
    'rr_y': -0.0988675829003773,
}
self_position = calibration_to_xywh(calibration)

for frame_index, track_frame, painter in ((first_i, first_frame, augmented_painter), (second_i, second_frame, sparse_painter)):
    pose = {'ts': track_frame['tracker']['current_pose']['ts'], 'x': 0., 'y': 0., 'teta': 0.}

    if track_frame['tracker']['free_tile_centers']:
        cells = [track_frame['tracker']['target_point']] + track_frame['tracker']['free_tile_centers']
        transformation = (track_frame['tracker']['current_pose']['x'], track_frame['tracker']['current_pose']['y'], track_frame['tracker']['current_pose']['teta'])
        transformed_cells = revert_transformation_to_cloud(cells, transformation)
        free_cells = transformed_cells[1:]
        target_point = transformed_cells[0]
    else:
        free_cells = None
        target_point = None

    painter.update_image(
        [pose],
        track_frame['lidar']['augmented_points'],
        self_position,
        free_cells=free_cells,
        target_point=target_point,
        image_text=str(frame_index),
        only_nearby_meters=only_nearby_meters
    )
        
dashboard.redraw()

In [8]:
# sys.path.append('/usr/local/lib/python3.6/dist-packages/python_pcl_ndt-0.2-py3.6-linux-x86_64.egg')

In [9]:
raw_result = [
    second_frame['tracker']['current_pose'][k] - first_frame['tracker']['current_pose'][k]
    for k in ['x', 'y', 'teta']
]
print(raw_result)

[0.0, 0.0, 0.0]


In [10]:
diff_image = widgets.Image(format='jpeg', width=640, height=640)
display(diff_image)

Image(value=b'', format='jpeg', height='640', width='640')

In [11]:
def get_dashboard_image(image, lidar_points1, lidar_points2, plot_size=1.0):
    figsize = figsize_from_image_size(image)
    fig = Figure(figsize=figsize)
    canvas = FigureCanvas(fig)

    ax = fig.add_subplot(plot_size, plot_size, 1)
    ax.set_xlim(-only_nearby_meters, only_nearby_meters)
    ax.set_ylim(-only_nearby_meters, only_nearby_meters)

    lidar_x_points = [x[0] for x in lidar_points1]
    lidar_y_points = [x[1] for x in lidar_points1]
    ax.scatter(lidar_x_points, lidar_y_points, marker='o', s=5, c='blue')

    lidar_x_points = [x[0] for x in lidar_points2]
    lidar_y_points = [x[1] for x in lidar_points2]
    ax.scatter(lidar_x_points, lidar_y_points, marker='o', s=4, c='red')

    ax.grid(which='both', linestyle='--', alpha=0.5)
    
    canvas.draw()
    jpeg = BytesIO()
    canvas.print_jpg(jpeg)
    image.value = jpeg.getvalue()

get_dashboard_image(
    diff_image,
    apply_transformation_to_cloud(first_frame['lidar']['augmented_points'], [first_frame['tracker']['current_pose'][k] for k in ['x', 'y', 'teta']]),
    apply_transformation_to_cloud(second_frame['lidar']['augmented_points'], [second_frame['tracker']['current_pose'][k] for k in ['x', 'y', 'teta']]),
)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: MatplotlibDeprecationWarning: Passing non-integers as three-element position specification is deprecated since 3.3 and will be removed two minor releases later.
  


In [12]:
from python_pcl_ndt import python_pcl_ndt

In [13]:
print(first_frame['tracker']['current_pose'])
print(second_frame['tracker']['current_pose'])

{'x': 1.0, 'y': 2.0, 'teta': 3.0, 'ts': 1724168571.5121946}
{'x': 1.0, 'y': 2.0, 'teta': 3.0, 'ts': 1724168572.3364182}


In [14]:
python_pcl_ndt.align_clouds(
    first_frame['tracker']['current_pose'],
    first_frame['lidar']['augmented_points'],
    True  # fast
)
result = python_pcl_ndt.align_clouds(
    second_frame['tracker']['current_pose'],
    second_frame['lidar']['augmented_points'],
    True  # fast
)

print(result)

get_dashboard_image(
    diff_image,
    apply_transformation_to_cloud(first_frame['lidar']['augmented_points'], [first_frame['tracker']['current_pose'][k] for k in ['x', 'y', 'teta']]),
    apply_transformation_to_cloud(second_frame['lidar']['augmented_points'], [result[k] for k in ['x', 'y', 'teta']]),
)

{'x_error': -0.2191791534423828, 'y_error': -0.23264431953430176, 'teta_error': -0.007548418361693621, 'x': 1.2498164176940918, 'y': 2.199385404586792, 'teta': 2.9924516677856445}


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: MatplotlibDeprecationWarning: Passing non-integers as three-element position specification is deprecated since 3.3 and will be removed two minor releases later.
  


In [15]:
(1.-0.1, 2.-0.15, 3.-0.05)

(0.9, 1.85, 2.95)